In [4]:
# =============================================================================
# Annual Ozone, Temperature, Mortality, Population data in Japan 
# Python translation of your R script
# =============================================================================

import pandas as pd
import numpy as np
import glob
import os
import re
from pathlib import Path

#Dependent Variables: Annual Deaths from Heat-Related Causes
#Independent Variables: Ozone, Temperature, Mortality, Population

In [5]:
# Set base directory
BASE_DIR = Path(r"/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2")

# Example: join subfolders and files
ozone_file_path = BASE_DIR / "Air_Pollutant" / "Oxidant_txt" / "TD19760613.txt"

print(ozone_file_path)

/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD19760613.txt


In [ ]:
# =============================================================================
# OZONE DATA
# =============================================================================

# Japanese → English column mapping
jp_to_en = {
    "測定年度": "year_recorded",
    "都道府県名": "prefecture",
    "市区町村コード": "city_code",
    "市区町村名": "city_name",
    "測定局名": "station_name",
    "用途地域名": "use_zone",
    "昼間の１時間値が0.06ppmを超えた日数(日)": "days_over_0_06",
    "昼間の１時間値が0.06ppmを超えた時間数(時間)": "hours_over_0_06",
    "昼間の１時間値の最高値(ppm)": "max_hourly_ppm",
    "昼間の日最高１時間値の年平均値(ppm)": "mean_annual_daytime_peak_ppm",
}

def year_from_filename(path):
    """Extract year from filename like TD19760613.txt (R: substr 3,6)."""
    fname = os.path.basename(path)
    return int(fname[2:6])

def read_tokyo_file(path):
    """Read ozone CSV with shift-jis/CP932 encoding."""
    return pd.read_csv(path, encoding="cp932")

def summarise_wards(path):
    raw = read_tokyo_file(path)
    keep_cols = [c for c in raw.columns if c in jp_to_en]
    df = raw[keep_cols].copy()

    # Rename to English
    df = df.rename(columns=jp_to_en)

    # city_code as string
    if "city_code" in df.columns:
        df["city_code"] = df["city_code"].astype(str)

        # keep city codes where 3rd char == "1"
        df = df[df["city_code"].str[2:3] == "1"]

    # filter use_zone == "住"
    if "use_zone" in df.columns and ("住" in df["use_zone"].dropna().values):
        df = df[df["use_zone"] == "住"]

    # If empty → return NA values
    if df.shape[0] == 0:
        yr = year_from_filename(path)
        return pd.DataFrame({
            "year": [yr],
            "mean_days_over_0_06": [np.nan],
            "mean_hours_over_0_06": [np.nan],
            "mean_max_hourly_ppm": [np.nan],
            "mean_annual_daytime_peak_ppm": [np.nan]
        })

    yr = year_from_filename(path)
    # Ensure numeric conversion for all relevant columns
    df["days_over_0_06"] = pd.to_numeric(df["days_over_0_06"], errors="coerce")
    df["hours_over_0_06"] = pd.to_numeric(df["hours_over_0_06"], errors="coerce")
    df["max_hourly_ppm"] = pd.to_numeric(df["max_hourly_ppm"], errors="coerce")
    df["mean_annual_daytime_peak_ppm"] = pd.to_numeric(df["mean_annual_daytime_peak_ppm"], errors="coerce")
    
    out = pd.DataFrame({
        "year": [yr],
        "mean_days_over_0_06": [df["days_over_0_06"].mean()],
        "mean_hours_over_0_06": [df["hours_over_0_06"].mean()],
        "mean_max_hourly_ppm": [df["max_hourly_ppm"].mean()],
        "mean_annual_daytime_peak_ppm": [df["mean_annual_daytime_peak_ppm"].mean()]
    })
    return out

In [7]:
# ---- apply to all files ----
ozone_dir = BASE_DIR / "Air_Pollutant" / "Oxidant_txt"
print("Looking in:", ozone_dir)
print("Exists?", ozone_dir.exists())
print("Contents:", list(ozone_dir.iterdir()) if ozone_dir.exists() else "Folder not found")

# glob needs a string pattern
ozone_files = sorted(glob.glob(str(ozone_dir / "TD*.txt")))
print("Ozone files found:", ozone_files)

if ozone_files:
    ozone_list = [summarise_wards(f) for f in ozone_files]
    df_results_ozone = pd.concat(ozone_list).sort_values("year").reset_index(drop=True)
else:
    print("No ozone files found to process.")
    df_results_ozone = pd.DataFrame(columns=[
        "year", "mean_days_over_0_06", "mean_hours_over_0_06",
        "mean_max_hourly_ppm", "mean_annual_daytime_peak_ppm"
    ])

Looking in: /Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt
Exists? True
Contents: [PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD20220613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD19980613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD19880613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD19750613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD19810613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD19910613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Air_Pollutant/Oxidant_txt/TD20140613.txt'), PosixPath('/Users/apple/Desktop/2025_research_paper_lam/OxTemp_An

In [16]:
print(df_results_ozone.head(52))

    year  mean_days_over_0_06  mean_hours_over_0_06  mean_max_hourly_ppm  \
0   1971                  NaN                   NaN             0.227778   
1   1972                  NaN                   NaN             0.251818   
2   1973                  NaN           1293.500000             0.254167   
3   1974            88.222222            333.222222             0.195556   
4   1975           127.333333            539.095238             0.239048   
5   1976           124.523810            477.333333             0.226667   
6   1977            54.238095            145.476190             0.151905   
7   1978            24.952381             72.285714             0.119048   
8   1979            17.333333             55.333333             0.120952   
9   1980            12.190476             32.619048             0.100000   
10  1981            11.619048             33.666667             0.096667   
11  1982            17.666667             59.142857             0.122857   
12  1983    

In [8]:
# ---- lag 1 year ----
df_results_ozone_lagged = df_results_ozone.copy()
for col in [
    "mean_days_over_0_06",
    "mean_hours_over_0_06",
    "mean_max_hourly_ppm",
    "mean_annual_daytime_peak_ppm"
]:
    df_results_ozone_lagged[col + "_lag1"] = df_results_ozone_lagged[col].shift(1)

# -----------------------------
# Preview results
# -----------------------------
print(df_results_ozone_lagged.head(52))

    year  mean_days_over_0_06  mean_hours_over_0_06  mean_max_hourly_ppm  \
0   1971                  NaN                   NaN             0.227778   
1   1972                  NaN                   NaN             0.251818   
2   1973                  NaN           1293.500000             0.254167   
3   1974            88.222222            333.222222             0.195556   
4   1975           127.333333            539.095238             0.239048   
5   1976           124.523810            477.333333             0.226667   
6   1977            54.238095            145.476190             0.151905   
7   1978            24.952381             72.285714             0.119048   
8   1979            17.333333             55.333333             0.120952   
9   1980            12.190476             32.619048             0.100000   
10  1981            11.619048             33.666667             0.096667   
11  1982            17.666667             59.142857             0.122857   
12  1983    

In [9]:
# =============================================================================
# TEMPERATURE DATA
# =============================================================================
import openpyxl

weather_file_path = BASE_DIR / "Weather" / "Weather_tokyo.xlsx"
df_yearly_temps = (
    pd.read_excel(weather_file_path, skiprows=6)
      .iloc[:, [0, 7, 8, 9, 10, 11]]
)

df_yearly_temps.columns = [
    "year", "daily_mean", "daily_max", "daily_min",
    "max_temp", "min_temp"
]

# clean numeric values
def clean_num(x):
    if pd.isna(x): return np.nan
    return float(str(x).replace("]", ""))

for col in ["daily_mean", "daily_max", "daily_min", "max_temp", "min_temp"]:
    df_yearly_temps[col] = df_yearly_temps[col].map(clean_num).round(1)

df_yearly_temps = df_yearly_temps[
    (df_yearly_temps["year"].notna()) & (df_yearly_temps["year"] <= 2024)
]

# add lag1 variables
df_yearly_temps_lagged = df_yearly_temps.copy()
for col in ["daily_mean", "daily_max", "daily_min", "max_temp", "min_temp"]:
    df_yearly_temps_lagged[col + "_lag1"] = df_yearly_temps_lagged[col].shift(1)

print(df_yearly_temps_lagged.head())

   year  daily_mean  daily_max  daily_min  max_temp  min_temp  \
0  1876        13.6       18.7        8.3      35.6      -9.2   
1  1877        14.2       19.1        8.9      34.9      -4.8   
2  1878        13.8       18.3        9.6      35.1      -7.6   
3  1879        14.6       19.4       10.0      33.9      -5.5   
4  1880        14.1       19.2        9.1      33.2      -6.8   

   daily_mean_lag1  daily_max_lag1  daily_min_lag1  max_temp_lag1  \
0              NaN             NaN             NaN            NaN   
1             13.6            18.7             8.3           35.6   
2             14.2            19.1             8.9           34.9   
3             13.8            18.3             9.6           35.1   
4             14.6            19.4            10.0           33.9   

   min_temp_lag1  
0            NaN  
1           -9.2  
2           -4.8  
3           -7.6  
4           -5.5  


In [10]:
# =============================================================================
# MORTALITY & POPULATION DATA
# =============================================================================

def read_ipss_table(path):
    """Auto-detect starting line, like R version."""
    # read raw lines
    with open(path, "r") as f:
        lines = f.readlines()

    hdr_line = None
    data_line = None

    for i, ln in enumerate(lines):
        if re.match(r"^\s*Year\s+Age\s+Female\s+Male\s+Total\s*$", ln):
            hdr_line = i
            break

    for i, ln in enumerate(lines):
        if re.match(r"^\s*\d{4}\s+\S+\s+[-0-9.]+\s+[-0-9.]+\s+[-0-9.]+\s*$", ln):
            data_line = i
            break

    start = hdr_line if hdr_line is not None else data_line
    if start is None:
        raise ValueError(f"Could not detect table start in: {path}")

    # pandas read from detected line
    if hdr_line is not None:
        df = pd.read_table(path, skiprows=hdr_line, sep=r"\s+")
    else:
        df = pd.read_table(path, skiprows=data_line, sep=r"\s+",
                           names=["Year", "Age", "Female", "Male", "Total"])

    # clean Age
    df["Age"] = df["Age"].astype(str).str.strip()
    df["Age"] = df["Age"].str.replace(r"\s+", "", regex=True)
    return df


def collapse_age(age):
    age = age.replace(" ", "")
    if age == "0": return "0"
    if age == "1-4": return "1-4"
    if age in ["5-9", "10-14"]: return "5-14"
    if age in ["15-19", "20-24"]: return "15-24"
    if age in ["25-29","30-34","35-39","40-44"]: return "25-44"
    if age in ["45-49","50-54","55-59","60-64"]: return "45-64"
    if age in ["65-69","70-74"]: return "65-74"
    if age in ["75-79","80-84"]: return "75-84"
    if age in ["85-89","90-94"]: return "85-94"
    if age in ["95-99","100-104","105-109","110+"]: return "95+"
    return None


def aggregate_by_year_sex(path, year_min=1947, year_max=2023):
    df = read_ipss_table(path)
    df = df[(df["Year"] >= year_min) & (df["Year"] <= year_max)]

    df["age_group"] = df["Age"].apply(collapse_age)
    df = df[df["age_group"].notna()]

    female = df.groupby(["Year", "age_group"])["Female"].sum().reset_index()
    female = female.pivot(index="Year", columns="age_group", values="Female").reset_index()

    male   = df.groupby(["Year", "age_group"])["Male"].sum().reset_index()
    male   = male.pivot(index="Year", columns="age_group", values="Male").reset_index()

    return {"female": female, "male": male}


# load deaths + population
deaths_file_path = BASE_DIR / "Mortality" / "IPSS" / "Deaths_5x1.txt"
population_file_path = BASE_DIR / "Mortality" / "IPSS" / "Population5.txt"

deaths_ts = aggregate_by_year_sex(deaths_file_path)
pop_ts    = aggregate_by_year_sex(population_file_path)


# --- combine into long form ---
age_levels = ["0","1-4","5-14","15-24","25-44","45-64","65-74","75-84","85-94","95+"]
sex_levels = ["Female", "Male"]

def make_long(df, sex, metric):
    df_long = df.melt(id_vars="Year", var_name="age_group", value_name=metric)
    df_long["sex"] = sex
    df_long = df_long.rename(columns={"Year": "year"})
    return df_long

def combine_four(deaths_ts, pop_ts):
    d_f = make_long(deaths_ts["female"], "Female", "Deaths")
    d_m = make_long(deaths_ts["male"],   "Male",   "Deaths")

    p_f = make_long(pop_ts["female"], "Female", "Population")
    p_m = make_long(pop_ts["male"],   "Male",   "Population")

    df_f = d_f.merge(p_f, on=["year","age_group","sex"])
    df_m = d_m.merge(p_m, on=["year","age_group","sex"])

    df = pd.concat([df_f, df_m])
    df["age_group"] = pd.Categorical(df["age_group"], categories=age_levels, ordered=True)
    df["sex"] = pd.Categorical(df["sex"], categories=sex_levels, ordered=True)
    return df.sort_values(["year","sex","age_group"])


df_long_ts = combine_four(deaths_ts, pop_ts)

In [11]:
# =============================================================================
# VEHICLE DATA
# =============================================================================
veh_dir = BASE_DIR / "Vehicles"

# -----------------------------
# Find all Excel files in the folder
# -----------------------------
from pathlib import Path

veh_dir = BASE_DIR / "Vehicles"

# Recursive, include all Excel formats, ignore temporary files
excel_files = [f for f in veh_dir.rglob("*.xls*") if not f.name.startswith("~$")]

print(f"Found {len(excel_files)} Excel files.")
for f in excel_files:
    print(f)

Found 52 Excel files.
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005hm.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005ii.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005je.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/r5c6pv000000v5aq.xlsx
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005ei.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005dm.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005fe.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005ga.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005iy.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicles/e49tph00000005f6.xls
/Users/apple/Desktop/2025_research_paper_lam/OxTemp_Annual_2/Vehicl

In [12]:
# Function to convert Japanese era year to Western year
def convert_japanese_year(cell):
    if not isinstance(cell, str):
        cell = str(cell)
    # Western year YYYY年
    m = re.search(r"(\d{4})年", cell)
    if m:
        return int(m.group(1))
    # Japanese era
    m = re.search(r"(昭和|平成|令和)(\d+)", cell)
    if m:
        era, y = m.groups()
        y = int(y)
        if era == "昭和":
            return 1926 + y - 1
        elif era == "平成":
            return 1989 + y - 1
        elif era == "令和":
            return 2019 + y - 1
    return None

In [13]:
# -----------------------------
# Loop over files and extract info
# -----------------------------
records = []

for f in excel_files:
    try:
        # Skip temp files
        if f.name.startswith("~$"):
            continue

        # Read Excel with correct engine
        if f.suffix.lower() == ".xls":
            df_excel = pd.read_excel(f, sheet_name=0, header=None, engine="xlrd")
        else:
            df_excel = pd.read_excel(f, sheet_name=0, header=None, engine="openpyxl")
        df_str = df_excel.astype(str)

        # Find Tokyo row
        tokyo_row_idx = None
        for r in range(df_str.shape[0]):
            if "東京" in df_str.iloc[r, :].values:
                tokyo_row_idx = r
                break

        # Extract all vehicle counts in that row
        vehicle_count = None
        place = "Tokyo"
        
        if tokyo_row_idx is not None:  
            for c in range(df_str.shape[1]):
                cell = df_str.iloc[tokyo_row_idx, c]
                num_cell_clean = re.sub(r"[^\d]", "", str(cell))
                if num_cell_clean.isdigit():
                    vehicle_count = int(num_cell_clean)
                    # Heuristic: passenger cars are often in the range of thousands to millions     
         # Extract year: first cell with 年 or Japanese era
        year = None
        for r in range(df_str.shape[0]):
            for c in range(df_str.shape[1]):
                year = convert_japanese_year(df_str.iloc[r, c])
                if year is not None:
                    break
            if year is not None:
                break

        # Add record
        records.append({
            "filename": f.name,
            "filepath": str(f),
            "year": year,
            "place": place,
            "vehicles": vehicle_count
        })

    except Exception as e:
        print(f"Error reading {f.name}: {e}")

# Combine into DataFrame
df_vehicle_count = pd.DataFrame(records).sort_values("year").reset_index(drop=True)
print(df_vehicle_count.head(60))

                 filename                                           filepath  \
0    e49tph000000059u.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
1    e49tph00000005a2.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
2    e49tph00000005aa.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
3    e49tph00000005ai.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
4    e49tph00000005aq.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
5    e49tph00000005ay.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
6    e49tph00000005be.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
7    e49tph00000005bm.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
8    e49tph00000005bu.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
9    e49tph00000005c2.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
10   e49tph00000005ca.xls  /Users/apple/Desktop/2025_research_paper_lam/O...   
11   e49tph00000005ci.xls  /Users/apple/

In [14]:
# =============================================================================
# MERGE Mortality + Temperature + Ozone + Vehicles (Tokyo)
# =============================================================================

df_mort_ozone_temp_veh = (
    df_long_ts
    .merge(df_yearly_temps_lagged, on="year", how="left")
    .merge(df_results_ozone_lagged, on="year", how="left")
    .merge(df_vehicle_count, on="year", how="left")
)

# Filter years 1978–2022
df_mort_ozone_temp_veh = df_mort_ozone_temp_veh[
    (df_mort_ozone_temp_veh["year"] >= 1978) &
    (df_mort_ozone_temp_veh["year"] <= 2022)
]

# Check
print(df_mort_ozone_temp_veh.head())

# Save to a specific directory
output_path = BASE_DIR / "Processed" / "df_mort_ozone_temp_veh.pkl"
df_mort_ozone_temp_veh.to_pickle(output_path)

     year age_group   Deaths     sex  Population  daily_mean  daily_max  \
620  1978         0   512.69  Female    77688.82        16.1       20.0   
621  1978       1-4   170.16  Female   342693.45        16.1       20.0   
622  1978      5-14   157.11  Female   814168.25        16.1       20.0   
623  1978     15-24   259.40  Female   898877.05        16.1       20.0   
624  1978     25-44  1602.38  Female  1999960.53        16.1       20.0   

     daily_min  max_temp  min_temp  ...  mean_max_hourly_ppm  \
620       12.6      36.3      -2.0  ...             0.119048   
621       12.6      36.3      -2.0  ...             0.119048   
622       12.6      36.3      -2.0  ...             0.119048   
623       12.6      36.3      -2.0  ...             0.119048   
624       12.6      36.3      -2.0  ...             0.119048   

     mean_annual_daytime_peak_ppm  mean_days_over_0_06_lag1  \
620                      0.029952                 54.238095   
621                      0.029952     